In [85]:
import pandas as pd
import numpy as np
import seaborn as sns
import treeswift
import math
import re

In [86]:
%load_ext autoreload
%autoreload 2
    
import helpers.utils
from helpers.utils import build_summary_df
from helpers.utils import plot_genotype_confidence
from helpers.utils import clustermap_genos
from helpers.utils import distdict_to_df, leaf_pairs, get_geno_dict
from helpers.utils import im_ehd, empirical_site_dists, ehd, sm_ehd, pair_metrics
from helpers.utils import plot_concordance_scatterplot, plot_concordance_distribution
from helpers.utils import plot_state_counts, report_genotype_call_stats, save_df_to_pdf
from helpers.utils import plot_correlation, plot_bl_variance, add_internal_labels, branch_table
from helpers.utils import plot_tree_3d, edge_ratio_table, plot_genotypecall_summary

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Remember not to re-run the process petracer code... it reshuffles the cell names!

I subset to clone 2 after I assigned the target_idx, so I have to remap the character_idx and target_idx. 

In [87]:
import sys
sys.path.insert(0, "/Users/gc3045/git/fast-laml/scripts") 
import euclidean_solver as es

In [88]:
inputs_basename = "/Users/gc3045/git/laml2-experiments/real_data/PEtracer/inputs/"
lamlpro_prefix = "/Users/gc3045/git/laml2-experiments/real_data/PEtracer/runjobs/outputs_petracer_seq_validation_clone2/fastlaml_clone2.neighbor_joining"

In [89]:
lp_input_argmax = inputs_basename + "petracer_clone2_kde_character_matrix.csv"
bm_input_geno = inputs_basename + "petracer_clone2_petracer_genotypes.csv"
all_bm_input_geno = inputs_basename + "petracer_full_training.csv"
lp_map_geno = lamlpro_prefix + "_posterior_argmax.csv"

bm_treefile = inputs_basename + "trees/seq_validation/clone2.neighbor_joining.nwk"
lp_treefile = lamlpro_prefix + "_tree.newick"

In [90]:
lp_input_geno = inputs_basename + "/petracer_clone2_kde_scores.csv"

#### Read inputs

In [91]:
lp_input_argmax_df = pd.read_csv(lp_input_argmax)

In [92]:
lookup_codebook = pd.read_csv(inputs_basename + "/lookup_codebook.csv")

In [93]:
petracer_full_traindf = pd.read_csv(all_bm_input_geno)

In [94]:
lp_input_geno_df = pd.read_csv(lp_input_geno)

In [95]:
bm_input_geno_df = pd.read_csv(bm_input_geno)
lp_map_geno_df = pd.read_csv(lp_map_geno, skiprows=2, index_col=0)

In [96]:
# bm_input_geno_df['target_idx'].unique() didn't get mapped to enumerated values

unique_indices = sorted(bm_input_geno_df['target_idx'].unique())
lookup_map = {idx: i for i, idx in enumerate(unique_indices)}

# assign the mapped enumeration
bm_input_geno_df['lookup_target_idx'] = bm_input_geno_df['target_idx'].map(lookup_map)

In [97]:
bm_input_geno_df['seq_state'].isna().sum()

np.int64(0)

#### Inspect a single cell as sanity check

In [98]:
lp_map_geno_df.loc['4T1_preedited-1420']

character_0    -1
character_1    -1
character_2    -1
character_3    -1
character_4    -1
character_5    -1
character_6    -1
character_7    -1
character_8    -1
character_9    -1
character_10   -1
character_11   -1
character_12   -1
character_13   -1
character_14   -1
character_15    3
character_16    7
character_17    8
character_18    2
character_19    4
character_20    8
character_21    8
character_22    5
character_23    1
character_24   -1
character_25   -1
character_26   -1
character_27   -1
character_28   -1
character_29   -1
character_30    2
character_31    4
character_32    2
character_33   -1
character_34   -1
character_35   -1
character_36    3
character_37    2
character_38    6
character_39   -1
character_40   -1
character_41   -1
character_42   -1
character_43   -1
character_44   -1
character_45    7
character_46    1
character_47    1
character_48   -1
character_49   -1
character_50   -1
character_51   -1
character_52   -1
character_53   -1
character_54    2
character_

In [99]:
mask = bm_input_geno_df['cellBC'] == '4T1_preedited-1420'
bm_input_geno_df.loc[mask].sort_values('lookup_target_idx')

,cellBC,intID,clone,x,y,z,target_site,pet_state,seq_state,brightest_state,...,state3_prob,state4_prob,state5_prob,state6_prob,state7_prob,state8_prob,cassette_idx,target_idx,kde_pred_label,lookup_target_idx
8081,4T1_preedited-1420,intID1427,2,1425,2177,40,RNF2,ACAGT,ACAGT,ACAGT,...,7.698111,5.008416,5.090930,5.779882,5.081611,5.560175,5,15,2,15
901,4T1_preedited-1420,intID1427,2,1425,2177,40,HEK3,GCAAG,GCAAG,GCAAG,...,-5.152927,-1.668323,-1.311345,-2.174178,6.587819,-18.458325,5,16,7,16
4491,4T1_preedited-1420,intID1427,2,1425,2177,40,EMX1,GGACA,GGACA,GGACA,...,3.494815,1.620918,-2.026061,0.844089,1.779553,5.707909,5,17,8,17
8281,4T1_preedited-1420,intID1469,2,1425,2119,43,RNF2,ACAGT,ACAGT,ACAGT,...,5.634398,2.250000,2.152266,3.697743,2.489670,2.109755,6,18,1,18
1101,4T1_preedited-1420,intID1469,2,1425,2119,43,HEK3,CTCTC,CTCTC,CTCTC,...,-6.809893,-1.004419,-6.645847,-10.423876,-3.717405,-5.272059,6,19,4,19
4691,4T1_preedited-1420,intID1469,2,1425,2119,43,EMX1,GGACA,GGACA,GGACA,...,0.866125,-2.585614,-16.861392,-0.063258,-21.263863,4.247639,6,20,8,20
8485,4T1_preedited-1420,intID1584,2,1445,2153,27,RNF2,TTCCT,TTCCT,TTCCT,...,-2.038231,-13.583472,-13.277112,-5.663754,-13.129991,4.944660,7,21,8,21
1305,4T1_preedited-1420,intID1584,2,1445,2153,27,HEK3,CTTTG,CTTTG,CTTTG,...,-7.157326,-15.720284,5.059393,-10.946750,-7.079981,1.525774,7,22,5,22
4895,4T1_preedited-1420,intID1584,2,1445,2153,27,EMX1,ACAAT,ACAAT,ACAAT,...,-5.854251,5.093725,-13.899764,-4.929870,-4.622817,-5.324240,7,23,1,23
9034,4T1_preedited-1420,intID1882,2,1400,2134,22,RNF2,ACTTA,ACTTA,ACTTA,...,7.988790,5.368962,5.414478,5.808836,5.159503,5.550343,10,30,2,30


In [100]:
code_map = (lookup_codebook
            .rename(columns={'kde_pred_label':'code', 'kde_pred_string':'lp_string'})
            .copy())

# enforce types
code_map['target_idx'] = pd.to_numeric(code_map['target_idx'], errors='coerce').astype('Int64')
code_map['code']       = pd.to_numeric(code_map['code'], errors='coerce').astype('Int64')

# --- 2) Prepare lp_map_geno_df: wide -> long with integer target_idx ---
lp = lp_map_geno_df.copy()
if lp.index.name != 'node':
    lp.index.name = 'node'
lp.index = lp.index.astype(str)

# rename 'character_k' -> k (int)
col_map = {c: int(re.search(r'(\d+)', str(c)).group(1)) for c in lp.columns}
lp = lp.rename(columns=col_map)

lp_long = (lp
    .stack()
    .rename('code')
    .reset_index()
    .rename(columns={'node':'cell_name', 'level_1':'target_idx'})
)

lp_long['target_idx'] = pd.to_numeric(lp_long['target_idx'], errors='coerce').astype('Int64')
lp_long['code']       = pd.to_numeric(lp_long['code'], errors='coerce').astype('Int64')

# --- 3) Map numeric code -> string using lookup_codebook ---
lp_long = lp_long.merge(code_map[['target_idx','code','lp_string']],
                        on=['target_idx','code'], how='left')

# treat -1 as missing explicitly
lp_long.loc[lp_long['code'].eq(-1), 'lp_string'] = 'missing'

# (optional) if any codes didn’t find a mapping, label as 'unknown'
lp_long['lp_string'] = lp_long['lp_string'].fillna('unknown')

# --- 4) Long or wide outputs ---
# Long (cell_name, target_idx, lp_string)
lp_long_out = lp_long[['cell_name','target_idx','lp_string']].copy()

# Wide (per-row table like your example, columns are target_idx)
lp_wide_out = (lp_long_out
               .pivot(index='cell_name', columns='target_idx', values='lp_string')
               .sort_index(axis=1))


In [101]:
mask = ~lp_long_out['cell_name'].astype(str).str.startswith('internal', na=False)
lp_out = lp_long_out.loc[mask].copy()

In [102]:
lp_out

,cell_name,target_idx,lp_string
0,4T1_preedited-1420,0,missing
1,4T1_preedited-1420,1,missing
2,4T1_preedited-1420,2,missing
3,4T1_preedited-1420,3,missing
4,4T1_preedited-1420,4,missing
...,...,...,...
13915,4T1_preedited-7108,55,GCGCC
13916,4T1_preedited-7108,56,CCCTA
13917,4T1_preedited-7108,57,TTCCT
13918,4T1_preedited-7108,58,GCGCC


In [103]:
tmp_bm = bm_input_geno_df[['cellBC', 'lookup_target_idx', 'pet_state', 'seq_state']]
tmp_bm.columns = ['cell_name', 'target_idx', 'pet_state', 'seq_state']

In [104]:
tmp_bm

,cell_name,target_idx,pet_state,seq_state
0,4T1_preedited-1153,1,CTCTC,CTCTC
1,4T1_preedited-1215,1,CTCTC,CTCTC
2,4T1_preedited-1280,1,CTCTC,CTCTC
3,4T1_preedited-1335,1,CTCTC,CTCTC
4,4T1_preedited-1349,1,CTCTC,CTCTC
...,...,...,...,...
10765,4T1_preedited-8440,57,TTCCT,TTCCT
10766,4T1_preedited-8557,57,TTCCT,TTCCT
10767,4T1_preedited-8611,57,TTCCT,TTCCT
10768,4T1_preedited-8623,57,TTCCT,TTCCT


In [105]:
tmp_bm['seq_state'].isna().sum()

np.int64(0)

In [106]:
merged = tmp_bm.merge(lp_out, on=["cell_name", "target_idx"], how="inner")

In [107]:
merged

,cell_name,target_idx,pet_state,seq_state,lp_string
0,4T1_preedited-1153,1,CTCTC,CTCTC,CTCTC
1,4T1_preedited-1215,1,CTCTC,CTCTC,CTCTC
2,4T1_preedited-1280,1,CTCTC,CTCTC,CTCTC
3,4T1_preedited-1335,1,CTCTC,CTCTC,CTCTC
4,4T1_preedited-1349,1,CTCTC,CTCTC,CTCTC
...,...,...,...,...,...
10765,4T1_preedited-8440,57,TTCCT,TTCCT,TTCCT
10766,4T1_preedited-8557,57,TTCCT,TTCCT,TTCCT
10767,4T1_preedited-8611,57,TTCCT,TTCCT,TTCCT
10768,4T1_preedited-8623,57,TTCCT,TTCCT,TTCCT


In [108]:

# 3) Accuracy: ignore ONLY rows where both are 'missing'
both_missing = (merged["seq_state"] == "missing") & (merged["lp_string"] == "missing")
agree        = (merged["seq_state"] == merged["lp_string"]) & (~both_missing)
evaluated    = ~both_missing

n_eval   = int(evaluated.sum())
n_agree  = int(agree.sum())
accuracy = (n_agree / n_eval) if n_eval else np.nan

print(f"Accuracy (lp_string vs seq_state), ignoring ONLY both-missing:")
print(f"  n_evaluated = {n_eval}")
print(f"  n_agree     = {n_agree}")
print(f"  accuracy    = {accuracy:.4f}" if n_eval else "  accuracy    = NA (no evaluable rows)")

# 4) Double-check: every LP-missing spot is BM-missing
lp_missing_only = (merged["lp_string"] == "missing") & (merged["seq_state"] != "missing")
n_lp_missing_only = int(lp_missing_only.sum())
if n_lp_missing_only == 0:
    print("✅ All LP-missing spots are also BM-missing.")
else:
    print(f"⚠️  Found {n_lp_missing_only} spots where LP is 'missing' but BM is not.")
    # optional: show a few offending rows
    display_cols = ["cell_name", "target_idx", "seq_state", "lp_string"]
    print(merged.loc[lp_missing_only, display_cols].head(10))

Accuracy (lp_string vs seq_state), ignoring ONLY both-missing:
  n_evaluated = 10770
  n_agree     = 9207
  accuracy    = 0.8549
✅ All LP-missing spots are also BM-missing.
